`pip install tensorflow`
`pip install tensorflow_io` (can't be installed via conda)
`pip install mlflow`
Do I get a problem with numpy? Tensorflow uses 1.23, mlflo installed 1.24?

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
import tensorflow_io as tfio
import os
import pathlib

#import numpy as np
import tensorflow.experimental.numpy as tnp
from tensorflow.python.ops import gen_audio_ops as audio_ops

#  enable NumPy behavior for TensorFlow:
tnp.experimental_enable_numpy_behavior()

import mlflow

In [6]:
sniffingDir = "data/train/Sniffing"
backgroundDir = "data/train/Background"
testSniffingDir = "data/test/Sniffing"
testBackgroundDir = "data/test/Background"

validation_fraction = 0.2

In [7]:
def generate_spectrogram(file_path, label):
    # does not scale as sniffing should be the same independent of background level
    audio_tensor = tfio.audio.AudioIOTensor(file_path, dtype=tf.int16)
    audio = tf.cast(audio_tensor[:], tf.float32)
    spectrogram = audio_ops.audio_spectrogram(audio,
                                              window_size=320,
                                              stride=160,
                                              magnitude_squared=True)
    spectrogram = tf.nn.pool(
        input=tf.expand_dims(spectrogram, -1),
        window_shape=[1, 6],
        strides=[1, 6],
        pooling_type='AVG',
        padding='SAME')
    spectrogram = tf.squeeze(spectrogram, axis=0)
    # Not sure whether the log is a good idea...
    spectrogram = tnp.log10(spectrogram + 1e-6)
    return spectrogram, label

def prepare_data(dir, value):
    filePath = os.path.join(dir, "*.wav")
    files = tf.data.Dataset.list_files(filePath)
    values = tf.zeros(len(files)) if value == 0 else tf.ones(len(files))
    data = tf.data.Dataset.zip((files, tf.data.Dataset.from_tensor_slices(values)))
    spectrogramData = data.map(generate_spectrogram)
    return spectrogramData

def generate_binary_dataset(trueDataDir, falseDataDir):
    trueData = prepare_data(trueDataDir, 1)
    falseData = prepare_data(falseDataDir, 0)
    combinedData = trueData.concatenate(falseData)
    combinedData = combinedData.cache()
    return combinedData.shuffle(buffer_size=combinedData.cardinality().numpy())

def split_validation(allData, fraction):
    numberDataSets = allData.cardinality().numpy()
    validationCount = round(numberDataSets * fraction)
    trainSet = allData.take(numberDataSets - validationCount)
    validationSet = allData.skip(numberDataSets - validationCount).take(validationCount)
    return trainSet, validationSet

def batch_prefetch(dataSet):
    dataSet = dataSet.batch(16)
    dataSet = dataSet.prefetch(8)
    return dataSet

def calc_accuracy(predictions, true_values):
    accuracy = sum(map(lambda x, y: x == y == 1, true_values, predictions))/sum(true_values)
    return accuracy

def representative_data_gen():
    repr_samples, repr_labels = train.as_numpy_iterator().next()
    yield [repr_samples]

In [8]:
data = generate_binary_dataset(trueDataDir=sniffingDir, falseDataDir=backgroundDir)
train, validation = split_validation(data, validation_fraction)
train = batch_prefetch(train)
validation = batch_prefetch(validation)
samples, labels = train.as_numpy_iterator().next()
inputShape = samples.shape[1:]
print(f"inputShape {inputShape}")

inputShape (99, 43, 1)


In [9]:
# https://github.com/atomic14/diy-alexa/blob/master/model/Train%20Model.ipynb
model = Sequential([
    Conv2D(4, 3,
           padding='same',
           activation='relu',
           kernel_regularizer=regularizers.l2(0.001),
           name='conv_layer1',
           #input_shape=(IMG_WIDTH, IMG_HEIGHT, 1)),
           input_shape=inputShape),
    MaxPooling2D(name='max_pooling1', pool_size=(2,2)),
    Conv2D(4, 3,
           padding='same',
           activation='relu',
           kernel_regularizer=regularizers.l2(0.001),
           name='conv_layer2'),
    MaxPooling2D(name='max_pooling2', pool_size=(2,2)),
    Flatten(),
    Dropout(0.2),
    Dense(
        40,
        activation='relu',
        kernel_regularizer=regularizers.l2(0.001),
        name='hidden_layer1'
    ),
    Dense(
        1,
        activation='sigmoid',
        kernel_regularizer=regularizers.l2(0.001),
        name='output'
    )
])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_layer1 (Conv2D)        (None, 99, 43, 4)         40        
                                                                 
 max_pooling1 (MaxPooling2D)  (None, 49, 21, 4)        0         
                                                                 
 conv_layer2 (Conv2D)        (None, 49, 21, 4)         148       
                                                                 
 max_pooling2 (MaxPooling2D)  (None, 24, 10, 4)        0         
                                                                 
 flatten (Flatten)           (None, 960)               0         
                                                                 
 dropout (Dropout)           (None, 960)               0         
                                                                 
 hidden_layer1 (Dense)       (None, 40)                3

In [10]:
hist = model.fit(train, epochs=30, validation_data=validation)

Epoch 1/30
1/1 [==============================] - 2s 2s/step - loss: 0.7910 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.9048 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 29ms/step - loss: 0.8164 - recall: 1.0000 - precision: 0.3333 - val_loss: 1.0724 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 26ms/step - loss: 0.9042 - recall: 1.0000 - precision: 0.3333 - val_loss: 0.5738 - val_recall: 1.0000 - val_precision: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 31ms/step - loss: 0.6279 - recall: 1.0000 - precision: 0.6667 - val_loss: 0.6238 - val_recall: 1.0000 - val_precision: 1.0000
Epoch 5/30
1/1 [==============================] - 0s 28ms/step - loss: 0.7372 - recall: 0.5000 - precision: 1.0000 - val_loss: 0.6128 - val_recall: 1.0000 - val_precision: 1.0000
Epoch 6/30
1/1 [==============================] - 0s 31ms/step - loss: 0.5954 - rec

In [11]:
testData = generate_binary_dataset(trueDataDir=testSniffingDir, falseDataDir=testBackgroundDir)
testData = batch_prefetch(testData)
list_predictions = []
list_true = []
for i in range(testData.cardinality().numpy()):
    X_test, y_test = testData.as_numpy_iterator().next()
    testResult = map(lambda x: 1 if x > 0.5 else 0, model.predict(X_test))
    list_predictions.extend(testResult)
    list_true.extend(y_test)
calc_accuracy(list_predictions,list_true)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: data/test/Sniffing\\*.wav'

In [12]:
savedModelDir = os.path.join(os.getcwd(), "sniffing_model/1/")
tf.saved_model.save(model, savedModelDir)
converter = tf.lite.TFLiteConverter.from_saved_model(savedModelDir)

# Post Training Quantization
# https://www.tensorflow.org/lite/performance/post_training_quantization
# Integer with float fallback (using default float input/output)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Integer only
# doesn't work here, as I have float32 input and want an output 0<= x <= 1
# atomic14_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# atomic14_converter.inference_input_type = tf.int8  # or tf.uint8
# atomic14_converter.inference_output_type = tf.int8

tflite_model = converter.convert()

tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Joerg\Documents\tmp\SniffingDetector\Detecting\NeuralNetwork\sniffing_model/1/assets


INFO:tensorflow:Assets written to: C:\Users\Joerg\Documents\tmp\SniffingDetector\Detecting\NeuralNetwork\sniffing_model/1/assets


43400

In [13]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
tinyResults = []
tinyExpected = []
for i in range(testData.cardinality().numpy()):
    X_test, y_test = testData.as_numpy_iterator().next()
    for j in range(len(X_test)):
        interpreter.set_tensor(input_details[0]['index'], [X_test[j]])
        interpreter.invoke()
        tflite_result = interpreter.get_tensor(output_details[0]['index'])
        tinyResults.append(1 if tflite_result[0][0] > 0.5 else 0)
        tinyExpected.append(y_test[j])

tinyAccuracy = calc_accuracy(tinyResults, tinyExpected)
print(tinyAccuracy)

NameError: name 'testData' is not defined

Run `xxd -i model.tflite > model_data.cc` in git bash